# Simple Model Testing
Upload your trained model and test dataset to calculate F1-score and recall

In [ ]:
# Install required packages
!pip install scikit-learn pandas numpy deap imbalanced-learn openpyxl joblib scipy

In [ ]:
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from sklearn.impute import SimpleImputer
import warnings
import joblib
from google.colab import files

# Optional imports with fallbacks
try:
    from imblearn.over_sampling import SMOTE
except ImportError:
    print("Warning: imbalanced-learn not available")
    
try:
    from deap import base, creator, tools, algorithms
except ImportError:
    print("Warning: deap not available")
    
try:
    from scipy.io import arff
except ImportError:
    print("Warning: scipy not available for ARFF files")

# Suppress warnings
warnings.filterwarnings('ignore')

In [ ]:
# Upload your trained model (.pkl file)
print("Upload your trained model (.pkl file):")
uploaded_model = files.upload()
model_filename = list(uploaded_model.keys())[0]

# Load the model (handle both simple models and complex model_info dictionaries)
try:
    # Try loading with joblib first (preferred method from your training code)
    model_data = joblib.load(model_filename)
    
    if isinstance(model_data, dict) and 'model' in model_data:
        # Complex model info dictionary from your training code
        model = model_data['model']
        selected_features = model_data.get('selected_feature_indices', None)
        scaler_from_model = model_data.get('scaler', None)
        print(f"Loaded complex model with metadata from '{model_filename}'")
        print(f"Selected features: {len(selected_features) if selected_features else 'All'}")
    else:
        # Simple model object
        model = model_data
        selected_features = None
        scaler_from_model = None
        print(f"Loaded simple model from '{model_filename}'")
        
except:
    # Fallback to pickle if joblib fails
    with open(model_filename, 'rb') as f:
        model_data = pickle.load(f)
    
    if isinstance(model_data, dict) and 'model' in model_data:
        model = model_data['model']
        selected_features = model_data.get('selected_feature_indices', None)
        scaler_from_model = model_data.get('scaler', None)
    else:
        model = model_data
        selected_features = None
        scaler_from_model = None
    
    print(f"Model '{model_filename}' loaded successfully with pickle!")

print(f"Model type: {type(model)}")

In [ ]:
# Upload your test dataset
print("Upload your test dataset (.csv, .xlsx, or .arff):")
uploaded_data = files.upload()
data_filename = list(uploaded_data.keys())[0]

# Load the dataset
if data_filename.endswith('.csv'):
    df = pd.read_csv(data_filename)
elif data_filename.endswith('.xlsx'):
    df = pd.read_excel(data_filename)
elif data_filename.endswith('.arff'):
    from scipy.io import arff
    data, meta = arff.loadarff(data_filename)
    df = pd.DataFrame(data)

print(f"Dataset '{data_filename}' loaded successfully!")
print(f"Dataset shape: {df.shape}")
print("\nFirst 5 rows:")
print(df.head())

In [ ]:
# Prepare the data
# Assume the last column is the target variable
X_test = df.iloc[:, :-1]
y_test = df.iloc[:, -1]

# Convert categorical target to numeric if needed
if y_test.dtype == 'object':
    y_test = pd.Categorical(y_test).codes

# Use the model's saved scaler if available, otherwise create new one
if scaler_from_model is not None:
    print("Using scaler from saved model")
    X_test_scaled = scaler_from_model.transform(X_test)
else:
    print("Creating new scaler for features")
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)

# Apply feature selection if the model was trained with specific features
if selected_features is not None:
    print(f"Applying feature selection: {len(selected_features)} out of {X_test_scaled.shape[1]} features")
    X_test_scaled = X_test_scaled[:, selected_features]
else:
    print("Using all features")

print(f"Final test features shape: {X_test_scaled.shape}")
print(f"Test labels shape: {y_test.shape}")

In [ ]:
# Make predictions
y_pred = model.predict(X_test_scaled)

# Calculate metrics
f1 = f1_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
accuracy = accuracy_score(y_test, y_pred)

# Display results
print("=" * 50)
print("MODEL PERFORMANCE RESULTS")
print("=" * 50)
print(f"F1-Score:  {f1:.4f}")
print(f"Recall:    {recall:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Accuracy:  {accuracy:.4f}")
print("=" * 50)